## Колокації

Одним із важливих аспектів будь-якої природної мови є сполучуваність слів. В англійській мові поняття сполучуваності дуже відчутне і присутнє у всіх мовних структурах: ми кажемо _"make a mistake",_ але _"do a favour";_ ми кажемо _"big surprise",_ але _"great anger";_ ми кажемо _"highly unlikely",_ але _"seriously wrong"._

У цьому завданні вам потрібно дослідити сполучуваність дієслів одного синонімного ряду з прислівниками. Наприклад, ми частіше кажемо _"love somebody dearly", "honor somebody highly",_ але _"admire somebody greatly"._

**Завдання:**
1. продовжте синонімний ряд дієслів: _"say", "tell", "speak", "claim", "communicate"_
2. напишіть функцію, яка знаходить у реченні дієслово (за складеним раніше синонімним рядом) і витягає усі можливі прислівники на "-ly", якими це дієслово керує
3. напишіть програму, яка знайде усі можливі прислівники для наших дієслів у [корпусі блогів](blog2008.txt)
4. на виході програма повинна видати десять найчастотніших прислівників для кожного дієслова (разом із частотою). Приклад виводу:
	```
	say: (loudly, 51), (silently, 45), (quietly, 10)
	tell: (quietly, 100), (loudly, 61), (seriously, 5)
	```
5. збережіть програму та результати аналізу на корпусі (вивід) у директорії з вашим іменем

Синоніми можна знайти у тезаурусах (http://www.thesaurus.com/, https://www.merriam-webster.com/thesaurus) чи [WordNet](http://wordnetweb.princeton.edu/perl/webwn).

Корпус блогів взятий з [Political Blog Corpora](http://www.cs.cmu.edu/~ark/blog-data/).

Ви можете використати будь-яку мову програмування та будь-яку NLP-бібліотеку.

P.S. Зверніть увагу на те, що у природній мові дієслова можуть мати різні форми, а прислівників може бути по декілька.



In [8]:
"""1. продовжте синонімний ряд дієслів: "say", "tell", "speak", "claim", "communicate"""

# Let's use Wordnet to find synonyms
from nltk.corpus import wordnet 

init_words = ['say', 'tell', 'speak', 'claim', 'communicate']
synonyms = set()

for word in init_words:
    for syn in wordnet.synsets(word):
        if syn.lemmas()[0].name() not in synonyms:
            synonyms.add(syn.lemmas()[0].name().lower())

print(synonyms)

# It's interesting to note that the final list also includes initial words even though
# they were not explicilty added to the resulting list. This has most likely happened
# due to synonymic cross-referencing of each input word.

{'suppose', 'tell', 'order', 'read', 'distinguish', 'title', 'commune', 'say', 'talk', 'state', 'address', 'convey', 'allege', 'call', 'speak', 'pronounce', 'assure', 'communicate', 'claim'}


In [53]:
import spacy
from pprint import pprint as pp

"""2. напишіть функцію, яка знаходить у реченні дієслово (за складеним раніше синонімним 
рядом) і витягає усі можливі прислівники на "-ly", якими це дієслово керує"""

# Tests 
s1 = "Speak softly and carry a big stick. You will communicate more efficiently."
s2 = "He vehemently alleged that GIF is pronounced as 'JIF'!"

# Init model
nlp = spacy.load('en_core_web_sm')


def find_collocs(verbs, text, model) -> list:
    """Find verb collocations in a text
    
    :rtype list of collocations as tuples [('v1', 'adv1'), ('v2', 'adv2')]"""
    
    # Tokenize!
    doc = model(text)
    verb_collocs = []
    
    # Scan tokens and look for eligible verbs
    for token in doc:
        if token.lemma_ in verbs:
            verb = token.lemma_ 
            
            # Search children of eligible verbs for adverbs
            for child in token.children:
                if child.pos_ == "ADV" and child.text[-2:] == 'ly':
                    advmod = child.lemma_
                    # Add collocation as a tuple to list
                    verb_collocs.append((verb, advmod))
    
    return verb_collocs
            
    
find_collocs(synonyms, s2, nlp)

[('allege', 'vehemently')]

In [79]:
DEBUG = False
IN_FILE = '../../../tasks/02-structural-linguistics/blog2008.txt'

def analyze(in_file, verbs):
    """3. напишіть програму, яка знайде усі можливі прислівники для наших дієслів у 
    [корпусі блогів](blog2008.txt)"""
    
    nlp = spacy.load('en_core_web_md')
    collocs = []
    colloc_freq = {}
    
    # DEBUG
    limiter = 0
    
    i = open(in_file, 'r')
    
    for line in i.readlines():
        line_collocs = find_collocs(verbs, line, nlp)
        collocs += line_collocs
        
        # DEBUG
        limiter += 1
        if limiter >= 500 and DEBUG:
            break
    
    for colloc in collocs:
        # Split collocation into verb and adverb, for easier comprehension
        verb, adv = colloc[0], colloc[1]
        
        # If the verb is not in the dict, create a new bucket
        if verb not in colloc_freq:
            colloc_freq[verb] = {}
        
        # If the bucket is there, fill it with tallies for adverbs
        if adv not in colloc_freq[verb]:
            colloc_freq[verb][adv] = 1
        else:
            colloc_freq[verb][adv] += 1
    
    return colloc_freq
    
cf = analyze(IN_FILE, synonyms)

pp(cf)

{'address': {'actually': 6,
             'adequately': 3,
             'aggressively': 1,
             'apparently': 1,
             'appropriately': 1,
             'barely': 1,
             'blunderingly': 1,
             'briefly': 2,
             'completely': 1,
             'comprehensively': 1,
             'concisely': 1,
             'conspicuously': 1,
             'contemptuously': 1,
             'correctly': 1,
             'definitively': 1,
             'deftly': 1,
             'directly': 16,
             'effectively': 1,
             'eloquently': 1,
             'exactly': 1,
             'explicitly': 1,
             'finally': 1,
             'forthrightly': 2,
             'fully': 2,
             'hastily': 1,
             'honestly': 1,
             'hopefully': 1,
             'immediately': 2,
             'indirectly': 1,
             'intelligently': 1,
             'memorably': 1,
             'merely': 1,
             'mostly': 1,
             'only': 1,


In [80]:
from collections import OrderedDict as od
import operator

def print_top_10(cf):
    """4. на виході програма повинна видати десять найчастотніших прислівників для 
    кожного дієслова (разом із частотою). Приклад виводу:
    
    say: (loudly, 51), (silently, 45), (quietly, 10)
    tell: (quietly, 100), (loudly, 61), (seriously, 5)"""
    
    for verb in cf.keys():
        print(verb+': ', end='')
        for adverb in cf[verb].keys():
            x = sorted(cf[verb].items(), key=lambda kv: kv[1], reverse=True)
        i = 0
        while i < len(x) and i < 11:
            print("({0}: {1})".format(x[i][0], x[i][1]), end=' ')
            i += 1
        print()
        
    
print_top_10(cf)

claim: (falsely: 66) (previously: 9) (repeatedly: 8) (recently: 4) (initially: 3) (actually: 3) (absurdly: 3) (credibly: 3) (incorrectly: 3) (publicly: 3) (laughably: 3) 
say: (recently: 77) (actually: 74) (repeatedly: 55) (simply: 46) (explicitly: 38) (basically: 36) (publicly: 36) (really: 31) (only: 25) (clearly: 23) (previously: 22) 
speak: (directly: 32) (publicly: 15) (only: 12) (fiercely: 12) (briefly: 9) (generally: 9) (openly: 8) (politically: 7) (loudly: 7) (recently: 7) (previously: 6) 
call: (actually: 16) (only: 12) (repeatedly: 11) (previously: 11) (rightly: 8) (famously: 8) (openly: 8) (consistently: 8) (publicly: 8) (recently: 7) (explicitly: 7) 
tell: (recently: 26) (reportedly: 14) (privately: 11) (finally: 10) (specifically: 9) (only: 9) (basically: 9) (actually: 8) (really: 8) (simply: 8) (repeatedly: 7) 
talk: (directly: 14) (really: 12) (only: 9) (openly: 7) (actually: 7) (publicly: 6) (repeatedly: 6) (personally: 5) (clearly: 5) (constantly: 5) (regularly: 5) 
di